In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# Install libraries
!pip install pandas transformers torch openpyxl

In [48]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
import pandas as pd


In [49]:
# Đường dẫn file CSV
csv_path = "/content/drive/MyDrive/NesReading/books.csv"

# Load file CSV
data = pd.read_csv(csv_path)

# Định dạng dữ liệu thành các cặp "input" và "response"
formatted_data = []
for _, row in data.iterrows():
    category = row['category']
    title = row['title']
    formatted_data.append({"input": f"Category: {category}", "response": title})

# Chuyển đổi thành DataFrame
formatted_data = pd.DataFrame(formatted_data)

# Hiển thị một vài hàng đầu tiên để kiểm tra dữ liệu
print("Dữ liệu sau khi định dạng lại:")
print(formatted_data.head())


Dữ liệu sau khi định dạng lại:
                  input                               response
0     Category: FICTION                       The Great Gatsby
1  Category: NONFICTION                               Educated
2     Category: SCIENCE                A Brief History of Time
3     Category: HISTORY  Sapiens: A Brief History of Humankind
4   Category: BIOGRAPHY                             Steve Jobs


In [50]:
# Tải tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Thêm padding token nếu chưa có
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Đặt độ dài tối đa
max_length = 50

# Tokenize input và response
formatted_data['input_ids'] = formatted_data['input'].apply(
    lambda x: tokenizer.encode(x, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt").squeeze()
)

formatted_data['labels'] = formatted_data['response'].apply(
    lambda x: tokenizer.encode(x, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt").squeeze()
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [51]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_ids = self.inputs[idx]
        labels = self.labels[idx]

        # Padding cho `input_ids` để khớp với `labels`
        if len(input_ids) < max_length:
            input_ids = torch.cat([input_ids, torch.full((max_length - len(input_ids),), tokenizer.pad_token_id)])

        return {
            "input_ids": input_ids,
            "labels": labels,
        }

# Tạo dataset từ các `input_ids` và `labels`
inputs = list(formatted_data['input_ids'])
labels = list(formatted_data['labels'])
dataset = CustomDataset(inputs, labels)


In [52]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,  # Tăng batch size nếu GPU cho phép
    num_train_epochs=10,             # Số epoch phù hợp với dữ liệu nhỏ
    save_steps=500,
    save_total_limit=2,
    report_to=["none"],             # Tắt logging của W&B
    fp16=True                       # Sử dụng FP16 nếu dùng GPU
)


In [53]:
# Tải mô hình pre-trained
model = AutoModelForCausalLM.from_pretrained(model_name)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Huấn luyện mô hình
trainer.train()


Step,Training Loss


TrainOutput(global_step=40, training_loss=1.8827507019042968, metrics={'train_runtime': 110.2269, 'train_samples_per_second': 1.361, 'train_steps_per_second': 0.363, 'total_flos': 1913794560000.0, 'train_loss': 1.8827507019042968, 'epoch': 10.0})

In [58]:
# Đặt mô hình ở chế độ đánh giá
model.eval()

# Hàm generate để tạo response từ mô hình
def generate_response(input_text):
    inputs = tokenizer.encode_plus(
        input_text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length
    )
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        output_ids = model.generate(
          input_ids,
          attention_mask=attention_mask,
          max_new_tokens=50,
          num_return_sequences=1,
          num_beams=5,       # Thử beam search để cải thiện đầu ra
          pad_token_id=tokenizer.eos_token_id,
          do_sample=True,
          temperature=0.7,
          top_p=0.9,
          repetition_penalty=1.2
)


    response = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return response


# Thử nghiệm
test_input = "The Great Gatsby"
response = generate_response(test_input)

print("Input:", test_input)
print("Response:", response)


Input: The Great Gatsby
Response: The Great Gatsby
